# GitHub
* Github API数据结构定义, https://developer.github.com/v3/

In [2]:

import (
	"encoding/json"
	"fmt"
	"net/http"
	"net/url"
	"strings"
	"time"
)

const (
	IssuesURL = "https://api.github.com/search/issues"
)

type IssueSearchResult struct {
	TotalCount int `json:"total_count"`
	Items      []*Issue
}

// Issue
type Issue struct {
	Number    int
	HTMLURL   string `json:"html_url"`
	Title     string
	State     string
	User      *User
	CreatedAt time.Time `json:"created_at"`
	Body      string    // Markdown格式
}

// User
type User struct {
	Login   string
	HTMLURL string `json:"html_url"`
}

// SearchIssue
func SearchIssue(terms []string) (*IssueSearchResult, error) {
	q := url.QueryEscape(strings.Join(terms, " "))
	queryUrl := IssuesURL + "?q=" + q
	fmt.Println(queryUrl)
	resp, err := http.Get(queryUrl)
	if err != nil {
		return nil, err
	}

	if resp.StatusCode != http.StatusOK {
		resp.Body.Close()
		return nil, fmt.Errorf("search query failed: %s", resp.Status)
	}

	var result IssueSearchResult
	if err := json.NewDecoder(resp.Body).Decode(&result); err != nil {
		resp.Body.Close()
		return nil, err
	}
	resp.Body.Close()
	return &result, nil
}

In [3]:
import (
	"fmt"
	"testing"
)

// 搜索GitHub中Issue
func TestSearchIssue(t *testing.T) {
	terms := []string{"HBase"}
	result, err := SearchIssue(terms)
	if err != nil {
		t.Error(err)
	}
	fmt.Printf("%d issues:\n", result.TotalCount)
	for _, item := range result.Items {
		fmt.Printf("#%-5d %9.9s %.55s\n",
			item.Number, item.User.Login, item.Title)
	}
}

%test

=== RUN   TestSearchIssue
https://api.github.com/search/issues?q=HBase
44459 issues:
#9        elliVM Implement write to HBase 
#140   stuBirdFl hbase support cell TTL
#7        elliVM Implement SQL to HBase data mapping
#6        elliVM Draft WIP: Provide replication feature from Teragrep Ma
#18    renovate[ Update dependency org.apache.hbase:hbase-server to v0.9
#17    renovate[ Update dependency org.apache.hbase:hbase-protocol to v0
#16    renovate[ Update dependency org.apache.hbase:hbase-hadoop2-compat
#15    renovate[ Update dependency org.apache.hbase:hbase-hadoop-compat 
#6019  flaming-a [EPIC] Support Hbase catalog
#107   JackShi14 OBKV Hbase Put compatible to hbase 1.x
#6     renovate[ Dependency Dashboard
#125      elliVM HBase database POC 
#124      elliVM Add HBase dependencies
#45    javaoffer support hbase jql
#1         yjwww How to adapt to HBase
#613   github-ac Maven 4 Test Results: hbase-thirdparty (4.0.0-rc-1)
#1893  github-ac Maven 4 Test Results: hbase-thirdpart